# SUJET SEUL 

Afficher le panneau latéral avec la table des matières

# BDLE TP1 IMDB avec Databricks

révision du 24-09-2021

## Préparation

Commencer par attacher le notebook à un cluster. Cela démarre les resources de calcul pour traiter les cellules du notebook. Une façon simple de démarrer un cluster est de demander à exécuter une cellule du notebook :

In [0]:
spark

On ajuste l'environnement d'exécution des requêtes à la taille du cluster (8 coeurs)

In [0]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

### Uploader les datasets de IMDB

In [0]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

Télécharger les fichiers de IMDB directement sur le cluster databricks, sans passer par votre ordinateur perso

In [0]:
%sh
mkdir -p /temp
wget --no-verbose  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/imdb/vldb2015/csvfiles_sample001.tgz -O /temp/csvfiles_sample001.tgz
cd /temp && tar zxf /temp/csvfiles_sample001.tgz
ls -lh /temp

Définir le nom du dossier contenant les fichiers csv de IMDB

In [0]:
imdb_dir = "/FileStore/tables/imdb/"
print ("le dossier pour les fichiers de IMDB est:", imdb_dir)

le dossier pour les fichiers de IMDB est: /FileStore/tables/imdb/

Copier les fichiers de IMDB dans DBFS

In [0]:
dbutils.fs.mkdirs(imdb_dir)
dbutils.fs.cp("file:/temp/csvfiles_sample001", imdb_dir, recurse=True)

Liste des fichiers IMDB :

In [0]:
display(dbutils.fs.ls(imdb_dir))

Cette étape est **FACULTATIVE**

Ne **pas** faire cette étape si vous avez déjà importé les données du TP dans votre espace de stockage Databricks.

Télécharger l'archive contenant les fichiers IMDB.
Dans PUBLIC_DATASET https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4
aller dans le dossier imdb/vldb2015 et télécharger le fichier csvfiles_sample001.tgz

Lien direct :
https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/imdb/vldb2015&files=csvfiles_sample001.tgz

Extraire les fichiers csv de l'archive

Menu Data -> Add Data
Sélectionner un nouveau dossier pour _DBFS target directory_  /FileStore/tables/**imdb**
puis uploader tous les fichiers csv dans ce directory.

## Définir les tables de la base IMDB

La table Title affecte un identifiant id à un film

In [0]:
schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """

title = spark.read.csv(path = imdb_dir + "title.csv", schema = schema_title).persist()
title.createOrReplaceTempView("Title")
display(title)

id,title,imdb_index,kind_id,production_year,imdb_id,phonetic_code,episode_id,season_nr,episode_nr,series_years,md5sum
7532,"Beautiful, Married & Missing",null,7,2009,null,B3145,7383,null,null,null,890adf0e8ff5ba69b8f6dc7b0dce84e7
160794,Self-Awareness as a Soldier! Strength Lies in the Pure Heart,null,7,1995,null,S4165,160772,1,38,null,eb32612a744424c3b79a8959c48aa89b
20232,Baby Manning,null,7,1998,null,B152,20037,null,null,null,825389d4e35f378f950238020942782e
16604,The Next First Lady?/Here Comes the Girls/Toni Morrison,null,7,1999,null,N2316,15711,31,20,null,6385686401009aa7277c38cea7db489c
189974,(#1.6),null,7,2000,null,null,189966,1,6,null,2b97f31e49e4cea3da14e1d1292915d2
7034,(2001-04-26),null,7,2001,null,null,6845,null,null,null,b5773a56b3c21479ceada92cb3840612
63988,(2010-04-09),null,7,2010,null,null,63983,null,null,null,0f95b8ecd807c9e10b5181ffab8b6bf7
128719,(1999-03-03),null,7,1999,null,null,128714,null,null,null,0b40df9322c04d540aa496bf719eb265
12434,New Best Friend,null,7,2013,null,N1231,12433,1,3,null,9002eef5ad46236af79c0c7be159668b
109725,(2010-01-04),null,7,2010,null,null,109693,null,null,null,a3ee822c5e48f29313f30f674c5e9e13


La table Kind_type

In [0]:
#============
# Kind_Type
#============
schema_kind_type = "id INT, kind STRING"

kind_type = spark.read.csv(path = dir + "kind_type.csv", schema = schema_kind_type).persist()

kind_type.createOrReplaceTempView("Kind_Type")
#kind_type.printSchema()
display(kind_type)

La table Info_type indique la nature d'un attribut d'information. 
Sert dans Movie_Info, Person_Info

In [0]:
#==============
# Info_Type
#==============
schema_info_type = "id INT, info STRING"

info_type = spark.read.csv(path = dir + "info_type.csv", schema = schema_info_type).persist()

info_type.createOrReplaceTempView("Info_Type")
#info_type.printSchema()

display(info_type)

La table Movie_info

In [0]:
#==============
# Movie_Info
#==============
schema_movie_info = """
    id int,
    movie_id int,
    info_type_id int,
    info string,
    note string
"""

movie_info = spark.read.csv(path = dir + "movie_info.csv", schema = schema_movie_info).persist()

movie_info.createOrReplaceTempView("Movie_Info")
#movie_info.printSchema()

display(movie_info)

La table Cast_Info

In [0]:
#==============
# Cast_Info
#==============
schema_cast_info = """
  id int,
  person_id int,
  movie_id int,
  person_role_id int,
  note string,
  nr_order int,
  role_id int
"""

cast_info = spark.read.csv(path = dir + "cast_info.csv", schema = schema_cast_info).persist()

cast_info.createOrReplaceTempView("Cast_Info")
#cast_info.printSchema()

display(cast_info)

La table Role_type

In [0]:
#=============
# Role_Type
#=============
schema_role_type = "id INT, role STRING"

role_type = spark.read.csv(path = dir + "role_type.csv", schema = schema_role_type).persist()

role_type.createOrReplaceTempView("Role_Type")
#role_type.printSchema()

display(role_type)

La table Name affecte un identifiant id à une personne

In [0]:
#==============
# Name 
#==============
schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name = spark.read.csv(path = dir + "name.csv", schema = schema_name).persist()

name.createOrReplaceTempView("Name")
#name.printSchema()

display(name)

La table Person_Info

In [0]:
#==============
# Person_Info
#==============
schema_person_info = """
  id int,
  person_id int,
  info_type_id int,
  info string,
  note string
"""

person_info = spark.read.csv(path = dir + "person_info.csv", schema = schema_person_info).persist()
person_info.createOrReplaceTempView("Person_Info")

display(person_info)

La table Movie_Companies

In [0]:
#==============
# Movie_Companies
#==============
schema_movie_companies = """
    id int,
    movie_id int,
    company_id int,
    company_type_id int,
    note string
"""

movie_companies = spark.read.csv(path = dir + "movie_companies.csv", schema = schema_movie_companies).persist()

movie_companies.createOrReplaceTempView("Movie_Companies")
#movie_companies.printSchema()

display(movie_companies)

La table Company_Name

In [0]:
schema_company_name = """
    id int,
    name string,
    country_code string,
    imdb_id int,
    name_pcode_nf string,
    name_pcode_sf string,
    md5sum string
"""

company_name = spark.read.csv(path = dir + "company_name.csv", schema = schema_company_name)
company_name.createOrReplaceTempView("Company_Name")
#company_name.printSchema()

display(company_name)

La table Company_type

In [0]:
#==============
# Company_Type
#==============
schema_company_type = "id INT, kind STRING"

company_type = spark.read.csv(path = dir + "company_type.csv", schema = schema_company_type).persist()
company_type.createOrReplaceTempView("Company_Type")
#company_type.printSchema()

display(company_type)

## Exemples de requêtes

Rmq : dans une cellule de code, possibilité de faire _Run All Above_ pour traiter toutes les cellules précédentes.

### Requêtes sur les films : fichier title

In [0]:
%sql
select *
from title

#### Nombre de films

In [0]:
%sql
select count(*) as nbFilms 
from Title

#### Les types de films

In [0]:
%sql
select distinct kind_id 
from Title 
order by kind_id

#### Nombre de films par type de film

In [0]:
%sql
select kind_id, count(*) as nb 
from Title 
group by kind_id 
order by nb desc

#### Nombre de film par années

In [0]:
%sql
select production_year, count(*) as nbFilms
from Title
where production_year is not null
group by production_year
order by nbFilms desc

#### Distribution d'un attribut avec describe()

In [0]:
filmPerYear = spark.sql("""
select production_year, count(*) as nb
from Title
where production_year is not null
group by production_year
order by nb desc
""")

description = filmPerYear.describe('nb')
display(description)

#### Visualiser le nombre de films par années

[Documentation MatplotLib](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot)

In [0]:
import matplotlib.pyplot as plt

t = filmPerYear.collect()

tabYear = [x.production_year for x in t]
tabNb = [x.nb for x in t]

plt.bar(tabYear, tabNb)
plt.xlabel('year')
plt.ylabel('nb films')
plt.title('films per year')
plt.show()

### Requêtes sur les types de films : kind_type

In [0]:
%sql
SELECT * 
FROM Kind_type

#### Jointure entre Title et Kind_Type

In [0]:
title.printSchema()

In [0]:
%sql
SELECT k.kind, t.title, t.production_year 
FROM Title t, Kind_Type k 
WHERE t.kind_id = k.id
ORDER BY t.production_year DESC

#### Nombre de films par type

In [0]:
%sql
SELECT k.kind, count(*) as nb
FROM Title t, Kind_Type k 
WHERE t.kind_id = k.id
GROUP BY k.kind
ORDER BY count(*) DESC

### Requêtes sur les rôles : cast_info

#### Une personne a un ou plusieurs roles dans un film

In [0]:
cast_info.printSchema()

In [0]:
cast_info.count()

#### Nombre de roles par personne

In [0]:
res2 = spark.sql("""

select person_id, count(*) as nbRoles
from Cast_Info
group by person_id
order by nbRoles desc

""")
res2.persist()
res2.show(5)

descrire la distribution de l'attribut nbRoles

In [0]:
d = res2.describe('nbRoles')
d.show()

In [0]:
#t2 = res2.sample(0.01).collect()
t2 = res2.collect()

tabNb = [x.nbRoles for x in t2]

plt.yscale('log')
plt.plot(tabNb) 
plt.xlabel('actor')
plt.ylabel('nb roles')

#plt.hist(tabNb)
plt.show()

plt.yscale('linear')
plt.plot(tabNb) 
plt.xlabel('actor')
plt.ylabel('nb roles')

plt.show()

#### Le nombre de rôles par type de film depuis 2010

In [0]:
res3 = spark.sql("""
SELECT g.kind, count(*) as nb_roles
FROM Cast_Info c, Title f, Kind_type g
WHERE c.movie_id = f.id AND f.kind_id = g.id
AND f.production_year > 2010
GROUP BY g.kind
ORDER BY nb_roles desc
""")
res3.show(5)

In [0]:
t3 = res3.collect()

tabNb = [x.nb_roles for x in t3]
tabKind = [x.kind for x in t3]

plt.yscale('log')
plt.bar(tabKind,tabNb) 
plt.xlabel('kind')
plt.ylabel('nb roles')

#plt.hist(tabNb)

plt.show()

## Autres traitements

### Extraction de la date de sortie _release_dates_

In [0]:
%sql
create or replace temp view TitleDetail as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
;

select * 
from TitleDetail t
order by  t.production_year desc, t.title
limit 100;

Mémoriser le résultat de TitleDetail pour éviter de le calculer plusieurs fois

In [0]:
%sql
cache table TitleDetail

Une oeuvre peut avoir les propriétés suivantes

#### Informations détaillées sur toutes les propriétés d'une personne

In [0]:
%sql
create or replace temp view NameDetail as
select n.id, n.name, n.gender, i.info as property, p.info as value
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
;
 
select * 
from NameDetail n
where n.name like 'Smi%'
order by n.name, n.property
limit 100;

Mémoriser le résultat de NameDetail pour éviter de le calculer plusieurs fois

In [0]:
%sql
cache table NameDetail;

Une personne peut avoir les propriétés suivantes

#### Structurer la date de sortie d'un film
sélection des infos sur la date de sortie

In [0]:
%sql
create or replace temp view Release_Date as
select t1.id, t1.title, t1.value as release_date
from TitleDetail t1
where t1.property = "release dates";

select * from Release_Date limit 3;

découper la chaine _release_date_  pour séparer  le pays et la date

In [0]:
%sql
create or replace temp view SplitDate as 
select r.id, r.title, split(release_date, ':') as splitDate
from Release_Date r;

select *  from SplitDate s limit 3;

In [0]:
%sql
create or replace temp view CountryDMY as 
select s.id, s.title, element_at(s.splitDate, 1) as country, split(element_at(s.splitDate, 2),' ') as DMY
from SplitDate s
;

select * from  CountryDMY limit 3;

extraire les attributs : day, month, year

In [0]:
%sql
-- à compléter

### Diverses requêtes

#### Exemple de jointure naturelle vs. jointure externe

Jointure naturelle: Seulement les films possédant la propriété _color info_

In [0]:
%sql
-- on sait que la référence info_type_id vaut 2 pour 'color info' dans Movie_Info

select t.id, t.title, m.info as color_info
from Title t, Movie_Info m
where t.id = m.movie_id 
and m.info_type_id = 2
order by id
limit 5;

#### Questions diverses
Quels sont tous les noms d'attributs en rapport avec une date ?

In [0]:
%sql
select *
from Info_type t
where t.info like '%date%'

Quelle est la syntaxe de la fonction **if** dans la clause select ?

In [0]:
%sql
select n.name, n.gender, if(gender='m', 'Mr' , if(gender='f', 'Mme', 'Inconnue' )) as civilite
from Name n
where name like '%isa';

# QUESTIONS

## Restructuration depuis la table NameDetail

Définir ces relations apportant des informations structurées sur les personnes : 
 - une table Nickname assez simple qui redéfini la propriété nick names en un nouvel attribut de personne
 - une table Birth_actor qui inclut la date, le lieu et le nom de naissance des personnes
 - une table Death avec le lieu et la cause du décès des personnes
 - une table SplitMagCover avec le nom du magazine dans lequel une personne a fait la couverture, le pays et la date de ladite couverture
 - une table Heights qui recense la taille des personnes de manière exploitable, en les reconvertissant en tailles en cm, alors que les entrées de la base sont très diverses (en plusieurs unités également)

Regardons d'abord la table NameDetail

In [0]:
%sql
select *
from NameDetail

Commençons par passer en revue les différentes propriétés des acteurs.
Afficher les valeurs distinctes de l'apptribut property

In [0]:
%sql
-- COMPLETER : 
--select ...
--from NameDetail

#### Nicknames
Penser à transformer la valeur nick names en nouvel attribut, et à supprimer quelques caractères superflus

définir Nicknames(id, Name, Gender, Nickname)

In [0]:
%sql 
--create or replace temp view Nicknames as
--select ............................. as Nickname
--from NameDetail
--where property = '....................'
;

--select *
--from Nicknames
--order by Nickname

#### Birth_Actor 
avec nom, date et lieu de naissance

Birth_Actor (id, name, birth date, birth_name, birth_place, day, month, year)

In [0]:
%sql

--create or replace temp view Birth_Actor as
--select .............as Birth_Name, 
--...... as Birth_Place, 
-- ......as Day_of_birth, 
-- as Month_of_birth, 
-- as Year_of_birth 
--from ( select ..................
--) as n,
--(    select ............
--) as d,
--(    select .....
--) as note
--where ................
--  and 
--  ........

--select *
--from Birth_Actor


#### Death
La table Death avec lieu et cause de la mort en séparant le champ *death note* 

Death(id, name, location_of_death, cause_of_death)

In [0]:
%sql
--create or replace temp view Death as
--select ............. 
--from NameDetail
--where property = '...........'
--;

--.....


--select *
--from Death;

#### SplitMagCover 
Definir tout d'abord *Magazine cover photo*

Magazine_cover_photo(id, name, Cover_from_Magazine, Country, Date)

In [0]:
%sql 
create or replace temp view Magazine_Cover_Photo as
select 
from NameDetail
where property = '....'
;

select *
from Magazine_Cover_Photo

puis definir la relation SplitMagCover avec nom du magazine, pays et date de couverture

SplitMagCover(id, name, cover_from_magazine, date)

In [0]:
%sql 
create or replace temp view SplitMagCover as 
select ........... as Cover_from_Magazine, 
........... as Country, ........... as Date
from Magazine_cover_photo
;

select *
from SplitMagCover

#### Height
avec la taille des acteurs, la difficulté réside dans le reformatage/conversion des tailles en cm, feet et inches, 

Height(id, name, height_in_cm)

In [0]:
%sql 
create or replace temp view Height as 
select .... as height_in_cm
from NameDetail
where property = '....'
;

select id, name, int(height_in_cm) as height_in_cm
from Heights
order by height_in_cm desc
limit 10

In [0]:
%sql -- confirmation que la base de données a une valeur bizarre pour Jack Douglass :
select id, name, value
from NameDetail
where property = 'height' and id = 413534

##  Restructuration depuis la table TitleDetail

Définir des relations sur les oeuvres : 
 - une table Anecdote assez simple qui redéfini la propriété trivia en un nouvel attribut d'une oeuvre
 - une table length avec la durée des oeuvres et un rapide histogramme représentant le gros des données
 - une table title_with_color qui défini le caractère en couleur ou noir et blanc d'une oeuvre
 - une table movie_rentals avec la valeur et la somme et la devise monétaire correspondante
 - une table title_filming_date avec les dates de début et de fin de tournage des oeuvres, structurées en jours, mois et années

In [0]:
%sql -- la table TitleDetail
select *
from TitleDetail

#### Anecdotes

Anecdote(id, title, fun_fact)

In [0]:
%sql -- Une première table avec un fun fact
create or replace temp view Anecdote as
select 
from TitleDetail
where property = 'trivia'
;

select * from Anecdote;

#### Durée d'un DVD

DVDLength(id, title, length)

In [0]:
%sql 
--Create or replace temp view DVDlengths as
--select distinct id, title, value as length
--from titledetail
--where property = 'LD length'
--order by .... desc
--;

--select *
--from DVDlengths

Distribution des durées: combien de films dont la durée est inférieur à 50' minutes, supérieure à 150' ?

In [0]:
# res3 = spark.sql("""
# SELECT .............. as length
# FROM titledetail
# WHERE property = 'LD length'
# ORDER by ....... desc
# """)
# res3.show(5)

# t = res3.collect()
# tab = [x.length for x in t]
# plt.hist(tab, bins = [50,75,100,125,150])
# plt.show()

#### Titres de films dont on connait l'indication de couleur ou de N&B

In [0]:
%sql 
create or replace temp view title_with_color as
select id, title, value as Color_or_BW
from TitleDetail
where property = 'color info'
;

select * from title_with_color;

#### Movie Rentals

Movie_rentals(id, title, rental_value, currency, country)

In [0]:
%sql -- 
--create or replace temp view Movie_rentals as
--select ........ as currency,
--...........  as country
--from TitleDetail
--where property = 'rentals'
--;

--select *
--from Movie_rentals

####  date de début et fin de tournage 

Title_filming_date(id, title, start_date, end_date)

In [0]:
%sql -- Cinquième table de
--create or replace temp view title_filming_date as
--select .... as Ending_date
--from titleDetail
--where property = 'filming dates'
--;

select *
from title_filming_date;

#### réorganiser ces dates par jour, mois et année de début et fin de tournage

Title_filming_date (id, title, start_day, start_month, start_year, end_day, end_month, end_year)

In [0]:
%sql 
--create or replace temp view title_filming_date as
--select id, title, 
--.......... as Starting_day,
--.......... as Starting_Month,
--.......... as Starting_Year,
--.......... as Ending_day,
--...........as Ending_Month,
--......... as Ending_Year

__from title_filming_date
__;

select * from title_filming_date;

#### Role H/F
nombre d'H et de F par profession, 
pourcentage H F par profession

RoleHF(role, nbH, nbF, Hpct, Fpct)